In [76]:
#Sytem
import sys,os
import numpy as np
import pandas as pd
import warnings
import time
import random
import IPython
warnings.filterwarnings('ignore')

#Visualidation
import sweetviz as sv
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

#EDA
import sweetviz as sv
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder



#model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from skopt import BayesSearchCV
import lightgbm as lgb



print('-'*40)

print("Python version: {}". format(sys.version))

print("pandas version: {}". format(pd.__version__))

print("matplotlib version: {}". format(matplotlib.__version__))

print("NumPy version: {}". format(np.__version__))


print("scikit-learn version: {}". format(sklearn.__version__))
print('-'*40)


----------------------------------------
Python version: 3.9.10 (main, Feb 22 2022, 13:54:07) 
[GCC 11.2.0]
pandas version: 1.3.5
matplotlib version: 3.5.1
NumPy version: 1.21.5
scikit-learn version: 1.1.1
----------------------------------------


In [38]:
Target = "Age"
PATH_LOG = "./Age_log.csv"
PATH_df = "./df"
ID = 00
SEED = 42

In [39]:
df_train = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")
log = pd.read_csv(PATH_LOG)
data = pd.concat([df_train,df_test])
print('-'*25)
print("Shape of Train Data\n[row :{},column :{}]".format(
    df_train.shape[0], df_train.shape[1]))
print('-'*25)
print("Shape of Test Data\n[row :{},column :{}]".format(
    df_test.shape[0], df_test.shape[1]))
print('-'*25)

-------------------------
Shape of Train Data
[row :891,column :12]
-------------------------
Shape of Test Data
[row :418,column :11]
-------------------------


In [40]:
data.head()

,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,0.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,0.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,0.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,1.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [41]:
data= data.drop(columns=["Perished"])

In [42]:
label = LabelEncoder()
ohe = OneHotEncoder()

In [43]:
data['Title'] = data['Name'].str.split(
        ", ", expand=True)[1].str.split(".", expand=True)[0]

title_names = (data.Title.value_counts() < 10)
data["Title"] = data["Title"].apply(lambda x: "other" if title_names.loc[x] == True else x)

In [44]:
faremean = data.Fare.mean()
data["Fare"].fillna(faremean,inplace=True)
data['FareBin'] = pd.qcut(data['Fare'], 10)
data['Fare_Code'] = label.fit_transform(data['FareBin'])

In [45]:
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,FareBin,Fare_Code
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,"(-0.001, 7.57]",0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,"(41.579, 78.02]",8
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,"(7.854, 8.05]",2
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,"(41.579, 78.02]",8
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,"(7.854, 8.05]",2


In [46]:
data["Embarked"].fillna('S',inplace=True)
data['Embarked_Code'] = label.fit_transform(data['Embarked'])

In [47]:
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,FareBin,Fare_Code,Embarked_Code
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,"(-0.001, 7.57]",0,2
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,"(41.579, 78.02]",8,0
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,"(7.854, 8.05]",2,2
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,"(41.579, 78.02]",8,2
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,"(7.854, 8.05]",2,2


In [48]:
data['Title_Code'] = label.fit_transform(data.Title)

In [49]:
data["Family"] = data.SibSp + data.Parch
data.replace({'Sex':{'male':0,'female':1}},inplace=True)

In [50]:
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,FareBin,Fare_Code,Embarked_Code,Title_Code,Family
0,1,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,"(-0.001, 7.57]",0,2,2,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,"(41.579, 78.02]",8,0,3,1
2,3,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,"(7.854, 8.05]",2,2,1,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S,Mrs,"(41.579, 78.02]",8,2,3,1
4,5,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S,Mr,"(7.854, 8.05]",2,2,2,0


In [51]:
drop_columns = ["Name","SibSp","Parch","Ticket","Cabin","Embarked","Title","FareBin"]
dataset = data.drop(columns=drop_columns)

In [52]:
dataset

,PassengerId,Pclass,Sex,Age,Fare,Fare_Code,Embarked_Code,Title_Code,Family
0,1,3,0,22.0,7.2500,0,2,2,1
1,2,1,1,38.0,71.2833,8,0,3,1
2,3,3,1,26.0,7.9250,2,2,1,0
3,4,1,1,35.0,53.1000,8,2,3,1
4,5,3,0,35.0,8.0500,2,2,2,0
...,...,...,...,...,...,...,...,...,...
413,1305,3,0,NaN,8.0500,2,2,2,0
414,1306,1,1,39.0,108.9000,9,0,4,0
415,1307,3,0,38.5,7.2500,0,2,2,0
416,1308,3,0,NaN,8.0500,2,2,2,0


In [53]:
train = dataset[dataset.Age.isnull() == False]
test = dataset[dataset.Age.isnull() == True]

In [114]:
len(test)

263

In [120]:
test.head()

,PassengerId,Pclass,Sex,Age,Fare,Fare_Code,Embarked_Code,Title_Code,Family
5,6,3,0,29.600531,8.4583,3,1,2,0
17,18,2,0,32.378513,13.0000,4,2,2,0
19,20,3,1,32.198237,7.2250,0,0,3,0
26,27,3,0,27.226836,7.2250,0,0,2,0
28,29,3,1,22.142105,7.8792,2,1,1,0


In [61]:
X = train.drop(columns=['PassengerId','Age']).values
y = train["Age"].values
X_test = test.drop(columns=["PassengerId","Age"]).values

In [115]:
len(X_test)

263

In [67]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [70]:
model = LinearRegression()
parameters = {
    'fit_intercept':[True,False],
    'normalize':[True,False],
    'copy_X':[True,False], 
}
grid = GridSearchCV(model,parameters, cv=5)
grid.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]})

In [73]:
grid.best_score_

0.3787962861179812

In [82]:
Modelname = "RandomForestRegressor"
rfc = RandomForestRegressor(criterion='mse')
CV = 5
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
param_grid = {
    'n_estimators':n_estimators,
    'max_features':max_features,
    'max_depth':max_depth,
    'min_samples_split':min_samples_split,
    'min_samples_leaf':min_samples_leaf,
    'bootstrap':bootstrap
}
print(param_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [83]:
Search = "BS"
rfc_cv = BayesSearchCV(estimator=rfc,search_spaces=param_grid,n_iter=100,cv=CV,verbose =2)
rfc_cv.fit(X,y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=2000; total time=   2.7s
[CV] END bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=2000; total time=   2.6s
[CV] END bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=2000; total time=   2.7s
[CV] END bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=2000; total time=   2.6s
[CV] END bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=2000; total time=   2.6s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1600; total time=   1.6s
[CV] END bootstrap=True, max_depth=30, max_feature

BayesSearchCV(cv=5, estimator=RandomForestRegressor(criterion='mse'),
              n_iter=100,
              search_spaces={'bootstrap': [True, False],
                             'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                           100, 110, None],
                             'max_features': ['auto', 'sqrt'],
                             'min_samples_leaf': [1, 2, 4],
                             'min_samples_split': [2, 5, 10],
                             'n_estimators': [200, 400, 600, 800, 1000, 1200,
                                              1400, 1600, 1800, 2000]},
              verbose=2)

In [84]:
best_model = rfc_cv.best_estimator_
print(best_model)
print('Train Score: {}'.format(round(best_model.score(X_train, y_train), 4)))
print('Valid Score: {}'.format(round(best_model.score(X_valid, y_valid), 4)))

RandomForestRegressor(criterion='mse', max_depth=10, max_features='sqrt',
                      min_samples_leaf=2, min_samples_split=10,
                      n_estimators=200)
Train Score: 0.5718
Valid Score: 0.5507


In [116]:
pred = rfc_cv.predict(X_test)

In [122]:
len(pred)


263

In [ ]:
lgb_train = lgb.Dataset(X_train,y_train)
lgb_eval = lgb.Dataset(X_valid,y_valid,reference=lgb_train)
params = {
    "objective":"binary",
    'metric':'auc',
    'varbosity':-1
}
evals_result = {}

model2 = lgb.train(params,lgb_train,valid_sets = lgb_eval,
num_boost_round=1000,
early_stopping_rounds =100,valid_names = ['train','eval'],evals_result = evals_result)

In [86]:
pred

array([29.60053123, 32.37851256, 32.19823716, 27.22683606, 22.14210526,
       28.0338338 , 38.94294384, 24.9003313 , 25.52784605, 30.28907744,
       28.89850107, 32.69650025, 24.9003313 , 23.46672737, 45.10894684,
       38.91355808,  7.53686844, 28.0338338 , 28.89850107, 23.83820585,
       28.89850107, 28.89850107, 28.0338338 , 27.71417465, 16.41238293,
       28.89850107, 34.06133117,  7.97722755, 24.67972153, 30.31333045,
       25.77666181, 13.82640003, 34.19300527, 44.60025557,  6.48157048,
       19.0924138 , 30.89976514, 41.84272579, 31.28283754, 34.06133117,
       24.9003313 , 28.12213273, 29.03881548, 28.0338338 ,  8.70432286,
       22.184538  , 14.79361625, 15.99498113, 30.31333045, 45.85713417,
       34.06133117, 24.9003313 , 46.37495311, 24.9003313 , 35.75645843,
       44.51547484, 38.91355808, 45.39051207, 24.9003313 , 28.80142729,
       28.2874019 , 28.89850107, 33.91835923, 28.12213273,  8.57633536,
       42.02772236, 28.0338338 , 29.38643668, 46.67386294, 27.22

In [125]:
preddata = pd.DataFrame(columns=['PassengerId', "Age"])
preddata['PassengerId'] = test['PassengerId']
preddata.Age = pred
preddata.head()

,PassengerId,Age
5,6,29.600531
17,18,32.378513
19,20,32.198237
26,27,27.226836
28,29,22.142105


In [132]:
list = dataset.PassengerId[dataset.Age.isnull()].tolist()

In [133]:
len(preddata)

263

In [134]:
len(list)

263

In [135]:
newdata = pd.concat([df_train,df_test])
newdata

,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,0.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,0.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,0.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,1.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [136]:
for idx in list:
    newdata["Age"][newdata.PassengerId == idx] = preddata["Age"][preddata.PassengerId == idx] 

0        1
0      892
1        2
1      893
2        3
      ... 
886    887
887    888
888    889
889    890
890    891
Name: PassengerId, Length: 1309, dtype: int64

0

In [141]:
newdata.head(20)

,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1.0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S
1,2,0.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C
2,3,0.0,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,0.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S
4,5,1.0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S
5,6,1.0,3,"Moran, Mr. James",male,29.600531,0,0,330877,8.4583,NaN,Q
6,7,1.0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,17463,51.8625,E46,S
7,8,1.0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,NaN,S
8,9,0.0,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.000000,0,2,347742,11.1333,NaN,S
9,10,0.0,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.000000,1,0,237736,30.0708,NaN,C


In [142]:
ndf_train = newdata[:len(df_train)]
ndf_test = newdata[len(df_train):]

In [146]:
ndf_train.to_csv("./data/train_alpha.csv", index_label=False)
ndf_test.to_csv("./data/test_alpha.csv", index_label=False)